In [48]:
import pandas
import re
from os import listdir
from os.path import isfile, join

path = '../data/new30/'   #change to directory with all files to be cleaned up

files = [f for f in listdir(path) if isfile(join(path, f)) and  not f.startswith('.')]  #scans directory and create list of files
files = ['Fife.xlsx']
new_list = list()

for i in range (0, len(files)):
    file = path + files[i]
    print file

    df = pandas.read_excel(file, index_col=None, header=None)    #reads excel file
    list_mrn = int(df.loc[2,2])    #finds mrn
    list_name = df.loc[0,2]        #finds name
    df = df.loc[6:len(df[0])]      #cuts head of dataframe up to first 'Name ... Date: ...' row

    for i, row in df.iterrows():
        m1 = m2 = m3 = False
        row = list(row)                                  #changes row type from object to list
        if (str(row[1]) != "nan"): list_meal = row[1]    #updates meal when appropriate    
        
        food = str(row[0])                               #finds food name in column 0
        if (food[0] == "^" or food[0] == "*"):           #item_nsc = item with no (^ or *) at the beginning
            list_food_nsc = food[1:]
        else:
            list_food_nsc = food
    
        #the script will filter out records containing the following texts
        p2 = ["Daily Average", "Daily Totals", "Daily Value", "Month Daily", "Percent Daily", "Week Daily"]
        p2 = p2 + ["Breakfast 1", "Breakfast 2", "Lunch 1", "Lunch 2", "Dinner 1", "Dinner 2"]
        p2 = p2 + ["Menu Item"]

        #the following reg expression describes the cell with patient name and date
        expression = (r'^[^,]+, [^ ]+( [^ ]+)*[ ]+Date:[ ]+[A-Za-z]+, (?P<month>[A-Za-z]+) (?P<day>[0-9]{1,2}), (?P<year>[0-9]{4})')
        p1 = re.compile (expression)
        m1 = p1.match (food)
    
        if m1:
            #updates date at new day
            list_date = m1.group ("month") + " " + m1.group ("day") + ", " + m1.group ("year")
    
        for item2 in p2:
            #checks for texts in p2
            if item2 in food:
                m2 = True
                break
            
        if food == "nan": m3 = True #checks for empty (nan) cells

        if not(m1 or m2 or m3):
            #if all controls on first column are negative -> adds new item to final list
            list_food = food
        
            expression = (r'(?P<value>[^ ]+)[ ]*(?P<unit>[a-zA-Z]+)')
            exp = re.compile (expression)
        
            portion_total = str(row[4])
            match = exp.match (portion_total)
        
            list_unit = match.group ("unit")            #defines unit of measurement
            list_por_tot = match.group ("value")        #defines amount of food ordered
        
            portion_eaten = str(row[5])
            match = exp.match (portion_eaten)
            list_por_eaten = match.group ("value")      #defines amount of food eaten
        
            new_line = [list_mrn, list_name, list_date, list_meal, list_food, list_food_nsc, list_unit, list_por_tot, list_por_eaten]
            new_line = new_line + row[7:18]
            print new_line[:]
            new_list.append(new_line)
#print new_list
#transforms list to dataframe and exports to excel
new_df = pandas.DataFrame (new_list, columns=["MRN", "Name", "Date", "Meal", "Food", "Food_NSC", "Unit", "Por_ordered", "Por_eaten", "Calories_kcal", "Proteing_g", "Fat_g", "Carbohydrates_g", "Fibers_g", "Sugars_g", "Sodium_mg"])
new_df.to_excel("../data/source/new30_all.xlsx")

#creates final column for unique foods consumed in the cohort
#unique_food_column = new_df["Food"].unique()
#unique_food_column.sort()
#column_df = pandas.DataFrame(unique_food_column)
#column_df.to_excel("../data/source/new30_all_unique_food.xlsx")

../data/new30/Fife.xlsx
[38060568, u'  FIFE, PATRICIA D', 'Aug 09, 2019', u'Dinner 2     (Menu: Low Microbial)', '2017 Pizza, Personal Pizza (4)', '2017 Pizza, Personal Pizza (4)', 'serv', '1', '1', u'412', u'49.6', u'11', u'889', u'66.5', u'3.4', u'4.1+', u'15.3']
[38060568, u'  FIFE, PATRICIA D', 'Aug 09, 2019', u'Dinner 2     (Menu: Low Microbial)', '2017 Toppings, Pepperoni for Hot', '2017 Toppings, Pepperoni for Hot', 'ounce', '2', '2', u'263', u'22.3', u'51', u'871', u'2.0', u'0.0', u'2.0', u'12.1']
[38060568, u'  FIFE, PATRICIA D', 'Aug 09, 2019', u'Dinner 2     (Menu: Low Microbial)', '2017 - Cup Of Ice  -', '2017 - Cup Of Ice  -', 'each', '1', '1', u'0', u'0.0', nan, u'5', u'0.0', nan, nan, u'0.0']
[38060568, u'  FIFE, PATRICIA D', 'Aug 09, 2019', u'Dinner 2     (Menu: Low Microbial)', '2017 Soda, Coca-Cola, Diet 12 oz', '2017 Soda, Coca-Cola, Diet 12 oz', 'each', '1', '1', u'0', u'0.0', u'0', u'40', u'0.1', u'0.0', u'0.0', u'0.0']
[38060568, u'  FIFE, PATRICIA D', 'Aug 10, 20

AssertionError: 16 columns passed, passed data had 17 columns